<a href="https://colab.research.google.com/github/bkovalevych/news_classification/blob/main/competition3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Define path and create df

In [ ]:
import numpy as np 
import pandas as pd 

import os
from pathlib import Path
root_path = '/content/drive/MyDrive/course4/NLP/ds_nlp_comp3/'
test_path = Path(root_path, "test_without_target.csv")
train_path = Path(root_path, "train.csv")
image_path = Path(root_path, "./images")

In [ ]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)


In [ ]:
print(train_df.shape)
train_df['images']

(16177, 5)


0                                                      NaN
1                 fd92a3dd-1109-49d8-8f5f-eeed72da22ef.png
2                 f4a284d0-9bb7-4910-8a62-a7bfd0ec29b6.png
3                                                      NaN
4        2cbb2339-6dd2-4000-a53c-225ec7aad892.png,d5233...
                               ...                        
16172                                                  NaN
16173                                                  NaN
16174                                                  NaN
16175    7770c665-825b-433a-b63e-f6e0ebe7f4cb.png,66866...
16176    272d0229-c589-4aca-aa0a-163e1b652461.png,17c8b...
Name: images, Length: 16177, dtype: object

In [ ]:
print(test_df.shape)
test_df.head()

(16183, 4)


,Id,title,text,images
0,0,Падіння економіки в Україні не є критичним - ЄБРР,Падіння економічного рівня в Україні не є крит...,NaN
1,3,Геймерам віддають безкоштовно шутер Rage 2 та ...,Шутер Rage 2 віддають безкоштовно в EGS / фото...,NaN
2,4,Записки из Луганска: привычка быть виноватыми ...,"""Ты печешь?"" – спрашивает у меня подруга, пере...","4530f95b-a014-424d-aefa-6f69fe82235a.png,2b8bc..."
3,6,Індекс Dow Jones вперше перевищив 30 тисяч пун...,На тлі новин щодо передачі влади у США та вакц...,NaN
4,7,Трамп і Меркель обговорили підтримку економічн...,Президент США Дональд Трамп і канцлерка Німечч...,NaN


In [ ]:
num_classes = train_df["source"].unique().shape[0]
num_classes

7

In [ ]:
train_df

,Id,title,text,images,source
0,1,Міністри Зеленського лишають мільйони пацієнті...,Понад півтора мільйони пацієнтів із тяжкими ді...,NaN,5
1,2,В АМКУ звертаються щодо незаконної державної д...,За 15 років державна підтримка виробництва ене...,fd92a3dd-1109-49d8-8f5f-eeed72da22ef.png,6
2,5,У Польщі виявили майже 16 тисяч нових хворих н...,У Польщі виявили 15 698 нових хворих на корона...,f4a284d0-9bb7-4910-8a62-a7bfd0ec29b6.png,2
3,9,Закриття міжбанку: гривня трохи ослабла,2 жовтня котирування гривні до долара на закри...,NaN,3
4,10,"У Москві помер український шахіст із бази ""Мир...",У п'ятницю низка російських ЗМІ повідомили про...,"2cbb2339-6dd2-4000-a53c-225ec7aad892.png,d5233...",0
...,...,...,...,...,...
16172,32366,"""Укрзалізниця"" просить Раду у бюджеті-2021 зак...","""Укрзалізниця"" закликає парламент закласти у д...",NaN,3
16173,32367,Сніг та 20-градусні морози протримаються до ви...,"У вівторок, 16 лютого, в Україні буде без істо...",NaN,2
16174,32368,"Україна хоче оподаткувати Google, Facebook та ...","Гетманцев розповів, як Україна оподатковуватим...",NaN,6
16175,32369,Лижі: як підготувати дитину до відпустки та че...,Якщо батькам подобається кататися на лижах – ц...,"7770c665-825b-433a-b63e-f6e0ebe7f4cb.png,66866...",5


In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(train_df.index.values, 
                                                  train_df["source"].values, 
                                                  test_size=0.10, 
                                                  random_state=42, 
                                                  stratify=train_df["source"].values)

train_df['data_type'] = ['not_set']*train_df.shape[0]

train_df.loc[x_train, 'data_type'] = 'train'
train_df.loc[x_val, 'data_type'] = 'val'

train_df.groupby(['source', 'data_type']).count()

Id  title  text  images
source data_type                           
0      train      1525   1525  1525    1457
       val         170    170   170     164
1      train      3084   3084  3084     830
       val         343    343   343      77
2      train       930    930   930     806
       val         103    103   103      92
3      train      2005   2005  2005     773
       val         223    223   223      85
4      train      2345   2345  2345     350
       val         261    261   261      52
5      train      1408   1408  1408     873
       val         156    156   156      95
6      train      3262   3262  3262    1406
       val         362    362   362     163

In [ ]:
!pip install transformers
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification


     |████████████████████████████████| 2.0MB 19.1MB/s 
     |████████████████████████████████| 3.2MB 48.8MB/s 
     |████████████████████████████████| 890kB 53.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=db9ceb3f157be4df7d9f25f976bd2caa378717005b0f0fa3788a529041100fb0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
def init_datasets(feature):
    tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', 
                                            do_lower_case=True)
                                            
    encoded_data_train = tokenizer.batch_encode_plus(
        train_df[train_df.data_type=='train'][feature].values, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        pad_to_max_length=True, 
        max_length=512, 
        return_tensors='pt'
    )

    encoded_data_val = tokenizer.batch_encode_plus(
        train_df[train_df.data_type=='val'][feature].values, 
        add_special_tokens=True, 
        return_attention_mask=True, 
        pad_to_max_length=True, 
        max_length=512, 
        return_tensors='pt'
    )


    input_ids_train = encoded_data_train['input_ids']
    attention_masks_train = encoded_data_train['attention_mask']
    labels_train = torch.tensor(train_df[train_df.data_type=='train']["source"].values)

    input_ids_val = encoded_data_val['input_ids']
    attention_masks_val = encoded_data_val['attention_mask']
    labels_val = torch.tensor(train_df[train_df.data_type=='val']["source"].values)

    # dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
    dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
    return dataset_val #dataset_train, dataset_val

dataset_val = init_datasets("text")
dataset_val_title = init_datasets("title")
dataset_train, dataset_val = init_datasets("text")
dataset_train_title, dataset_val_title = init_datasets("title")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `t

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased",
                                                      num_labels=num_classes,
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model_title = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased",
                                                      num_labels=num_classes,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)


dataloader_train_title = DataLoader(dataset_train_title, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation_title = DataLoader(dataset_val_title, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='macro')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

# Train model

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model = model.to(device)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    torch.save(model.state_dict(), f'{root_path}1024/BERT_epoch_{epoch}_acc_{val_f1}.model')
        
    
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Macro): {val_f1}')


Epoch 1
Training loss: 0.5103460712826575
Validation loss: 0.2637633844485204
F1 Score (Macro): 0.9341298884859274



Epoch 2
Training loss: 0.21189748146094683
Validation loss: 0.24891501479481665
F1 Score (Macro): 0.9441408859085504



Epoch 3
Training loss: 0.13948414097498504
Validation loss: 0.25657020928882973
F1 Score (Macro): 0.948104209633126



Epoch 4
Training loss: 0.09511226724449783
Validation loss: 0.2850198611578235
F1 Score (Macro): 0.93234504209171



Epoch 5
Training loss: 0.06368895114020023
Validation loss: 0.2765032487315756
F1 Score (Macro): 0.9507193671495765



Epoch 6
Training loss: 0.04576578945893593
Validation loss: 0.2621809248872174
F1 Score (Macro): 0.9499157434857197



Epoch 7
Training loss: 0.024865547111521363
Validation loss: 0.297559042787398
F1 Score (Macro): 0.9487733311698634



Epoch 8
Training loss: 0.012049396143363712
Validation loss: 0.3420770986286326
F1 Score (Macro): 0.9403064295319609


KeyboardInterrupt: ignored

# Choose best models

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased",
                                                      num_labels=num_classes,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model_title = BertForSequenceClassification.from_pretrained("bert-base-multilingual-uncased",
                                                      num_labels=num_classes,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)
model_title.to(device)

model.load_state_dict(torch.load('/content/drive/MyDrive/course4/NLP/ds_nlp_comp3/1024/BERT_epoch_6_acc_0.9499157434857197.model',
                                 map_location=torch.device('cpu')))
model_title.load_state_dict(torch.load('/content/drive/MyDrive/course4/NLP/ds_nlp_comp3/512_TITLE/BERT_epoch_5_acc_0.7033109698311587.model',
                                       map_location=torch.device('cpu')))



Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

<All keys matched successfully>

In [ ]:
model.eval()
model_title.eval()

predictions, true_vals = [], []
for batch, batch_title in zip(dataloader_validation, dataloader_validation_title):
    labels = batch[2]
    true_vals.append(labels)
    batch = tuple(b.to(device) for b in batch)
    batch_title = tuple(b.to(device) for b in batch_title)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
    inputs_title = {'input_ids': batch_title[0], 'attention_mask': batch_title[1]}
    with torch.no_grad():        
        outputs = model(**inputs)
        outputs_title = model_title(**inputs_title)
    logits = torch.argmax(outputs['logits'] + outputs_title['logits'], dim=1)    
    predictions.append(logits.detach().cpu().numpy())

predictions = np.concatenate(predictions, axis=0)
true_vals = np.concatenate(true_vals)



In [ ]:
f1 = f1_score(true_vals, predictions, average='macro')
f1

0.959802656329129

# Predict

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased', 
                                            do_lower_case=True)

encoded_data_test = tokenizer.batch_encode_plus(
    test_df["text"].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_test_title = tokenizer.batch_encode_plus(
    test_df["title"].values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)


input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']

input_ids_test_title = encoded_data_test_title['input_ids']
attention_masks_test_title = encoded_data_test_title['attention_mask']


dataset_test = TensorDataset(input_ids_test, attention_masks_test)
dataset_test_title = TensorDataset(input_ids_test_title, attention_masks_test_title)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
dataloader_test = DataLoader(dataset_test, 
                             sampler=SequentialSampler(dataset_test), 
                             batch_size=batch_size)

dataloader_test_title = DataLoader(dataset_test_title, 
                             sampler=SequentialSampler(dataset_test_title), 
                             batch_size=batch_size)

In [ ]:
model.eval()
model_title.eval()

predictions = []
for batch, batch_title in zip(dataloader_test, dataloader_test_title):
    batch = tuple(b.to(device) for b in batch)
    batch_title = tuple(b.to(device) for b in batch_title)
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}
    inputs_title = {'input_ids': batch_title[0], 'attention_mask': batch_title[1]}
    with torch.no_grad():        
        outputs = model(**inputs)
        outputs_title = model_title(**inputs_title)
    logits = torch.argmax(outputs['logits'] + outputs_title['logits'], dim=1)    
    predictions.append(logits.detach().cpu().numpy())

predictions = np.concatenate(predictions, axis=0)


In [ ]:
model.eval()
    
predictions = []
ids = []
for batch in dataloader_test:
    batch = tuple(b.to(device) for b in batch)
    
    inputs = {'input_ids': batch[0], 'attention_mask': batch[1]}

    with torch.no_grad():        
        outputs = model(**inputs)
        
    logits = torch.argmax(outputs['logits'], dim=1)    
    predictions.append(logits.detach().cpu().numpy())

predictions = np.concatenate(predictions, axis=0)

In [ ]:
result = pd.DataFrame({'Id': test_df["Id"].values, 'Predicted': predictions})

In [ ]:
result

,Id,Predicted
0,0,3
1,3,6
2,4,5
3,6,3
4,7,4
...,...,...
16178,32362,1
16179,32363,5
16180,32364,0
16181,32370,4


In [ ]:
result.to_csv(root_path + "smallest_loss.csv", index=False)